In [4]:
import pandas as pd
import os

from mecab import MeCab

from bertopic import BERTopic
from sklearn.feature_extraction.text import CountVectorizer

from hdbscan import HDBSCAN
from sentence_transformers import SentenceTransformer
from transformers import pipeline
from bertopic.representation import KeyBERTInspired

In [5]:
IN_DATA_PATH = './data/01_out'
OUT_DATA_PATH = './data/02_out'

in_data_files = []
in_docs = []
total_docs = []

# 폴더내 파일리스트를 가져온다.
input_files = os.listdir(IN_DATA_PATH)

# 입력 파일 로딩 RAW_DATA_PATH에 있는 파일을 DataFrame으로 읽어 드린다.
for in_file in input_files:
    print ("Loading file :%s ..."%in_file)
    df = pd.read_csv(os.path.join(IN_DATA_PATH,in_file), encoding='utf-8-sig')
    docs = df['generated_text'].tolist()
    in_data_files.append(df)
    in_docs.append(docs)
    total_docs.extend(docs)    


Loading file :umsun_A_2_sborder.csv ...
Loading file :umsun_B_sborder.csv ...
Loading file :umsun_clean_C.csv ...


In [6]:
class CustomTokenizer:
    def __init__(self, tagger):
        self.tagger = tagger
    def __call__(self, sent):
        word_tokens = self.tagger.morphs(sent)
        result = [word for word in word_tokens]
        return result



In [8]:
custom_tokenizer = CustomTokenizer(MeCab())
vectorizer = CountVectorizer(tokenizer=custom_tokenizer)

In [9]:
# Pre-calculate embeddings

#embedding_model = SentenceTransformer("sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens")

embedding_model = SentenceTransformer("snunlp/KR-SBERT-V40K-klueNLI-augSTS")
embeddings = embedding_model.encode(total_docs, show_progress_bar=True)

Batches:   0%|          | 0/625 [00:00<?, ?it/s]

In [10]:
# 차원축소
hdbscan_model = HDBSCAN(min_cluster_size=20, metric='euclidean', cluster_selection_method='eom', prediction_data=True)

In [11]:
# zero-shot
ko_classifier = pipeline(
    task='zero-shot-classification',
    model='MoritzLaurer/DeBERTa-v3-large-mnli-fever-anli-ling-wanli',
    device=0,
    # hypothesis_template='구매하는 이유는 {} 이다.',
)


In [12]:
CANDIDATE_LABELS = ['첨가물','성분','위험','아기','안심','칼로리','중위험','아이']

In [19]:
# [0] => A파일 , [1] => B파일 , [2] => C파일

docs = in_docs[0]

In [20]:
topic_model = BERTopic(
        embedding_model=embedding_model,
        hdbscan_model=hdbscan_model,
        vectorizer_model=vectorizer,
        nr_topics="auto", # 문서를 대표하는 토픽의 갯수
        # top_n_words=4,
        zeroshot_topic_list=CANDIDATE_LABELS,
        zeroshot_min_similarity=.5,
        # representation_model=ko_classifier,
        representation_model=KeyBERTInspired(),
        calculate_probabilities=True
)
	
topics, probs = topic_model.fit_transform(docs)
topic_model.get_topic_info()[:30]

,Topic,Count,Name,Representation,Representative_Docs
0,0,176,첨가물,"[첨가물이, 첨가물, 첨가물은, 첨가물도, 맛있는데, 맛있어요, 우유에, 먹어요, ...","[첨가물이 좀 있네요., 첨가물이 있네요., 첨가물이 없어서 좋아요.]"
1,1,109,성분,"[성분, 성분이, 성분은, 성분들이, 첨가물도, 음료에, 성분도, 알레르기라, 마시...","[저위험 성분이 있네요., 중위험 성분이 있어서 아쉬워요., 중위험 성분이 들어가 ..."
2,2,40,안심,"[무난해요, 무난합니다, 무난, 무난한, 괜찮네요, 괜찮았어요, 괜찮더라고요, 만족...","[무난해요., 무난해요., 가격 대비 괜찮아요.]"
3,3,28,아기,"[아기가, 아기, 아기도, 아기들, 먹였어요, 아가가, 먹어요, 아가, 아가들은, ...",[첫 요구르트로 사줘 봤는데 요구르트 당 함량은 어쩔 수 없나 봐요 근데 아기가 안...
4,4,8,칼로리,"[칼로리, 칼로리가, 먹어요, 다이어트할, 함량이, 간식으로, 먹으려고요, 맛있긴,...","[당류가 낮아서 다이어트할 때 간식으로 많이 먹어요., 넘나 맛있는 골드 키위 맛 ..."
5,5,8,중위험,"[중위험이, 중위험이네, 중위험이라서, 중위험, 사리면, 아쉽다, 있었군요, 있네요...","[중위험이 아쉽다., 중위험이 있었군요., 중위험이 있네요.]"
6,6,6,아이,"[아이, 아이보다, 아이가, 착하네요, 제품입니다, 아이에게, 좋아해요, 추천합니다...","[아이 배냇 추천합니다., 아이가 정말 좋아하는 제품입니다 제품이 착하네요., 아이..."
7,7,1,위험,"[저위험은, , , , , , , , , ]",[저위험은 없네요.]
8,8,97,-1_카레_먹어요_먹었어요_먹어야겠어요,"[카레, 먹어요, 먹었어요, 먹어야겠어요, 맛있는, 카레를, 맛이에요, 먹으면, 카...","[카레는 백세 카레 사다 먹어요., 이렇게 많은 갖가지 재료 가루들을 넣고도 맛없기..."
9,9,1064,0_맛있어요_먹어요_좋네요_맛있는,"[맛있어요, 먹어요, 좋네요, 맛있는, 좋아해요, 맛도, 좋아요, 있어요, 좋아해서...",[이거 맛있고 양도 많아서 자주 먹는데 위험 성분이 없어서 안심하고 먹을 수 있어서...


In [15]:
hierarchical_topics = topic_model.hierarchical_topics(docs)
tree = topic_model.get_topic_tree(hierarchical_topics)
print(tree)

TypeError: 'NoneType' object is not subscriptable

In [16]:
# Coherence
import gensim.corpora as corpora
from gensim.models.coherencemodel import CoherenceModel

# Preprocess documents for coherence evaluation
documents = pd.DataFrame({"Document": docs, "ID": range(len(docs)), "Topic": topics})
documents_per_topic = documents.groupby(['Topic'], as_index=False).agg({'Document': ''.join})
cleaned_docs = topic_model._preprocess_text(documents_per_topic.Document.values)

# Extract vectorizer and analyzer for BERTopic
vectorizer = topic_model.vectorizer_model
analyzer = vectorizer.build_analyzer()

# Extract features for topic coherence evaluation
words = vectorizer.get_feature_names_out()
tokens = [analyzer(doc) for doc in cleaned_docs]
dictionary = corpora.Dictionary(tokens)
corpus = [dictionary.doc2bow(token) for token in tokens]

# Extract words in each topic if they are non-empty and exist in the dictionary
topic_words = []
#topic_words = [list(zip(*model_2024_B_3.get_topic(topic)))[0] for topic in range(len(set(topics))-1)]
for topic in range(len(set(topics))-topic_model._outliers):
  words = list(zip(*topic_model.get_topic(topic)))[0]
  words = [word for word in words if word in dictionary.token2id]
  topic_words.append(words)
topic_words = [words for words in topic_words if len(words)>0]

# Evaluate Coherence
coherence_model = CoherenceModel(topics=topic_words, texts=tokens, corpus=corpus, dictionary=dictionary, coherence='c_v')
coherence = coherence_model.get_coherence()
print("Coherence Score: ", coherence)

NotFittedError: Vocabulary not fitted or provided